<a href="https://colab.research.google.com/github/nakib103/pancancer_classification/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'tensorflow == 2.1.0'

     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 450kB 34.9MB/s 
     |████████████████████████████████| 3.9MB 41.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ff5b603d7cbc41e97e731f40070834d8bd20b616d1401599d39793dc0f831278
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: 

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [ ]:
# import packages

import numpy as np
import pandas as pd
import pickle
import math

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LambdaCallback

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# run this cell to mount your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# load data and label

data = pd.read_pickle("/content/drive/My Drive/data/data_df.pkl")
label = pd.read_pickle("/content/drive/My Drive/data/label_df.pkl")

In [ ]:
def preprocess_data(data):
  # create data variable from loaded data
  data_log_transformed = data.apply(lambda x: np.log2(x + 1))

  return data_log_transformed

In [ ]:
def shuffle_data(data, label):
  ## same patient check
  # generate shuffled index
  shuffled_index = np.random.permutation(data.index)

  # shiffle data and label
  data_shuffled = data.reindex(shuffled_index)
  label_shuffled = label.reindex(shuffled_index)

  print(data_shuffled.head())
  print(label_shuffled.head())
  return data_shuffled, label_shuffled

In [ ]:
def label_pd_to_np(label):
  # create label variable from loaded label
  y = label.iloc[:].values

  for id_y, lab_y in enumerate(y):
    for id_class, lab_class in enumerate(classes):
      if lab_y[0] == lab_class:
        y[id_y] = id_class
        break
        
  y = y[:,0]
  y = to_categorical(y, num_classes=len(classes))

  return y

In [ ]:
# generate data and label
def generate_dataset(data, label):
 
  data = preprocess_data(data)
  data, label = shuffle_data(data, label)
  data = data.iloc[:].values
  label = label_pd_to_np(label)

  return data, label

In [ ]:
classes = ['GBM', 'OV', 'LUAD', 'LUSC', 'PRAD', 'UCEC', 'BLCA', 'TGCT', 'ESCA', 'PAAD', 'KIRP', 'LIHC', 'CESC', 'SARC', 'BRCA', 'THYM', 'MESO', 'COAD', 'STAD', 'SKCM', 'CHOL', 'KIRC', 'THCA', 'HNSC', 'LAML', 'READ', 'LGG', 'DLBC', 'KICH', 'UCS', 'ACC', 'PCPG', 'UVM']

X, Y = generate_dataset(data.iloc[:100], label.iloc[:100])

                              ?|100130426  ...  tAKR|389932
TCGA-OR-A5JJ-01A-11R-A29S-07          0.0  ...     2.192131
TCGA-PK-A5HB-01A-11R-A29S-07          0.0  ...     0.000000
TCGA-OR-A5KO-01A-11R-A29S-07          0.0  ...     1.266517
TCGA-OR-A5KW-01A-11R-A29S-07          0.0  ...     0.000000
TCGA-OR-A5JQ-01A-11R-A29S-07          0.0  ...     0.000000

[5 rows x 20531 columns]
                                0
TCGA-OR-A5JJ-01A-11R-A29S-07  ACC
TCGA-PK-A5HB-01A-11R-A29S-07  ACC
TCGA-OR-A5KO-01A-11R-A29S-07  ACC
TCGA-OR-A5KW-01A-11R-A29S-07  ACC
TCGA-OR-A5JQ-01A-11R-A29S-07  ACC


In [ ]:
X = np.expand_dims(X, axis=-1)
Y = np.expand_dims(Y, axis=1)
Tx = X.shape[1]
Ty = Y.shape[1]

In [ ]:
print(Y.shape)
# Y = np.append(Y, np.zeros((Y.shape[0], Y.shape[1], 2)), axis=-1)
# print(Y)

(100, 1, 33)


In [ ]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(256, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation("softmax", name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [ ]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

In [ ]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.  
# For the purposes of passing the automatic grader
# please do not modify this global variable.  This will be corrected once the automatic grader is also updated.
post_activation_LSTM_cell = GRU(n_s, return_state = True) # post-attention LSTM 
output_layer = Dense(len(classes), activation="softmax")

In [ ]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, gene_express_size, class_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    gene_express_size -- size of gene expression for a single gene (1)
    class_size -- size of class (33)

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, gene_express_size))
    # s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    # s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    a = Bidirectional(GRU(units=n_a, return_sequences=True))(X)
    a = Bidirectional(GRU(units=n_a, return_sequences=True))(a)
    print(type(a))
    print(a.shape)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        # context = one_step_attention(a, s)
        context = one_step_attention(a, c)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        # s, _, c = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        _, c = post_activation_LSTM_cell(inputs=context, initial_state=[c])

        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        # out = output_layer(s)
        out = output_layer(c)

        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    # model = Model(inputs=[X, s0, c0], outputs=outputs)
    model = Model(inputs=[X, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
model = model(Tx, Ty, n_a, n_s, 1, len(classes))

<class 'tensorflow.python.framework.ops.Tensor'>
(None, 20531, 64)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20531, 1)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20531, 64)    6720        input_1[0][0]                    
__________________________________________________________________________________________________
c0 (InputLayer)                 [(None, 64)]         0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20531, 64)    18816       bidirectional[0][0]              
______________________________________________________________________________________________

In [ ]:
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.get_layer("bidirectional").get_weights()))

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: (1e-8 * 10**(epoch/10)))
opt = SGD(lr=100)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
m = X.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Y.swapaxes(0,1))
history = model.fit([X, s0, c0], outputs, epochs=80, 
          batch_size=10, 
          validation_split=0.2, 
          # callbacks = [print_weights]
          )

Train on 80 samples, validate on 20 samples
Epoch 1/80
80/80 [==============================] - 56s 704ms/sample - loss: 533.2233 - accuracy: 0.6375 - val_loss: 708.9525 - val_accuracy: 0.8000
Epoch 2/80
80/80 [==============================] - 49s 615ms/sample - loss: 522.6667 - accuracy: 0.7125 - val_loss: 972.1904 - val_accuracy: 0.2000
Epoch 3/80
80/80 [==============================] - 49s 615ms/sample - loss: 708.5715 - accuracy: 0.6625 - val_loss: 164.9526 - val_accuracy: 0.8000
Epoch 4/80
80/80 [==============================] - 49s 613ms/sample - loss: 632.0714 - accuracy: 0.6375 - val_loss: 572.9528 - val_accuracy: 0.8000
Epoch 5/80
10/80 [==>...........................] - ETA: 38s - loss: 572.9528 - accuracy: 0.8000

In [ ]:
bidir = model.get_layer("bidirectional_1")
wght = bidir.get_weights()
wght